In [142]:
import pandas as pd
full_df = pd.read_csv("tabular-playground-series-sep-2022/train.csv", parse_dates=['date'])
full_df.head()
test_df = pd.read_csv("tabular-playground-series-sep-2022/test.csv", parse_dates=['date'])

In [143]:
full_df = full_df.assign(
    year = lambda df: df.date.map(lambda x: x.year),
    # month = lambda df: df.date.map(lambda x: x.month),
    season = lambda df: df.date.map(lambda x: x.quarter),
)

test_df = test_df.assign(
    year = lambda df: df.date.map(lambda x: x.year),
    # month = lambda df: df.date.map(lambda x: x.month),
    season = lambda df: df.date.map(lambda x: x.quarter),
)
full_df = pd.get_dummies(full_df)
test_df = pd.get_dummies(test_df)

In [144]:
from cgi import test
from sklearn.model_selection import train_test_split

# train_df, test_df = train_test_split(full_df, train_size=0.7, shuffle=False, random_state=42)
train_df = full_df
test_df = test_df
X = train_df.loc[:, "year":]
y = train_df["num_sold"]
X_test = test_df.loc[:, "year":]
# y_test = test_df["num_sold"]

In [145]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR

svm_model = make_pipeline(StandardScaler(), SVR(gamma='auto'))

In [146]:
from cgi import test
import numpy as np
from sklearn.model_selection import KFold

kf = KFold(5)
def get_oof(pipe, X_train, y_train, X_test):
    X_train = X_train.to_numpy()
    y_train = y_train.to_numpy()
    X_test = X_test.to_numpy()
    oof_train = np.zeros((train_df.shape[0], ))
    oof_test = np.zeros((test_df.shape[0], ))
    oof_test_skf = np.empty((5, test_df.shape[0]))

    for i, (train_index, test_index) in enumerate(kf.split(X_train)):
        kf_X_trian = X_train[train_index]
        kf_y_train = y_train[train_index]
        kf_X_test = X_train[test_index]

        model = pipe.fit(kf_X_trian, kf_y_train)
        oof_train[test_index] = model.predict(kf_X_test)
        oof_test_skf[i, :] = model.predict(X_test)
    
    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

# get_oof(svm_model, X, y, X_test)


In [147]:
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(n_estimators=100)
# get_oof(rf_model, X, y, X_test)

In [148]:
from sklearn.ensemble import GradientBoostingRegressor
gb_model = GradientBoostingRegressor(
    n_estimators=100, learning_rate=0.1, max_depth=1, random_state=0,
    loss='huber'
)
# get_oof(gb_model, X, y, X_test)

In [149]:
from sklearn.ensemble import AdaBoostRegressor
ab_model = AdaBoostRegressor(n_estimators=100)
# get_oof(ab_model, X, y, X_test)

In [150]:
from sklearn.ensemble import HistGradientBoostingRegressor
hgb_model = HistGradientBoostingRegressor(max_iter=100)

In [151]:
from sklearn.linear_model import Lasso
ls_model = Lasso(alpha=0.1)

In [152]:
models = {
    # "rf":rf_model, 
    "gb":gb_model, 
    "ab":ab_model, 
    "hgb":hgb_model,
    "ls": ls_model
}
train_ans = np.zeros((X.shape[0], len(models)))
test_ans = np.zeros((X_test.shape[0], len(models)))
for index, (model_name, model) in enumerate(models.items()):
    train_oof, test_oof = get_oof(model, X, y, X_test)
    train_ans[:,index] = train_oof[:,0]
    test_ans[:,index] = test_oof[:,0]

In [153]:
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

lm_model = LinearRegression()
lm_model.fit(train_ans, y)
lm_ans = lm_model.predict(test_ans)
pd.DataFrame(
    {
        "row_id": range(70128, 70128+len(lm_ans)),
        "num_sold": lm_ans
    }
).to_csv("submit.csv", index=False)

In [154]:
lm_model = LinearRegression()
lm_model.fit(X, y)
lm_ans = lm_model.predict(X_test)
mean_squared_error(y_test, lm_ans)

ValueError: Found input variables with inconsistent numbers of samples: [21039, 17520]